# QUICK START with REDIS SINGLE NODE

In [2]:
import redis 
# pip install redis

In [42]:
r = redis.Redis(host='127.0.0.1', port=55000)

### Delete all the datasets

In [43]:
r.flushall()

True

### Start saving key/values

In [4]:
r.set('foo', 'bar')
r.get('foo')

b'bar'

Re-using the same key makes overwritting

In [5]:
r.set('foo', 'bar2')
r.get('foo')

b'bar2'

### Pipelines

Pipelines are a subclass of the base Redis class that provide support for buffering multiple commands to the server in a single request. They can be used to dramatically increase the performance of groups of commands by reducing the number of back-and-forth TCP packets between the client and server.

In [6]:
pipe = r.pipeline()
pipe.set('luis', 'cuenca')
pipe.set('eduardo', 'barcelona')
pipe.set('maria','valencia')
pipe.get('luis')
pipe.get('maria')
pipe.execute()

[True, True, True, b'cuenca', b'valencia']

For ease of use, all commands being buffered into the pipeline return the pipeline object itself.  
Therefore calls can be chained like:

In [7]:
pipe = r.pipeline()
pipe.set('luis', 'cuenca').set('eduardo', 'barcelona').set('maria','valencia').get('luis').get('maria').execute()

[True, True, True, b'cuenca', b'valencia']

### Publish/Subscription

Redis-py includes a PubSub object that subscribes to channels and listens for new messages. Creating a PubSub object is easy.

In [9]:
import redis 
redis = redis.Redis(host='127.0.0.1', port=55000)
consumer = redis.pubsub()
consumer.subscribe('channel-movie', 'channel-cartoon')

At the begining we get the subscription ACK message

In [10]:
consumer.get_message()

{'type': 'subscribe', 'pattern': None, 'channel': b'channel-movie', 'data': 1}

In [11]:
consumer.get_message()

{'type': 'subscribe',
 'pattern': None,
 'channel': b'channel-cartoon',
 'data': 2}

In [12]:
consumer.channels

{b'channel-movie': None, b'channel-cartoon': None}

We can continue calling the get_message method and nothing is returned because there is no messages

In [13]:
consumer.get_message()

Let's now publish some data on the subscription channels. The returned value of the publish method is the number of subscribed channels affected by this publication

In [14]:
redis.publish('channel-movie', 'Movie1')

1

In [15]:
redis.publish('channel-movie', 'Movie2')

1

In [16]:
redis.publish('channel-movie', 'Movie3')

1

In [17]:
redis.publish('channel-cartoon', 'Cartoon1')

1

In [18]:
redis.publish('channel-cartoon', 'Cartoon2')

1

In [19]:
redis.publish('channel-cartoon', 'Cartoon3')

1

List Publish/Subscription channels:

In [20]:
redis.pubsub_channels()

[b'channel-movie', b'channel-cartoon']

To get the number of consumers subscribed at every channel. The number will be 0 if the consumer was closed

In [21]:
redis.pubsub_numsub('channel-movie', 'channel-cartoon')
# Return a list of (channel, number of subscribers)

[(b'channel-movie', 1), (b'channel-cartoon', 1)]

Let's read the messages

In [22]:
consumer.get_message()

{'type': 'message',
 'pattern': None,
 'channel': b'channel-movie',
 'data': b'Movie1'}

In [23]:
consumer.get_message()

{'type': 'message',
 'pattern': None,
 'channel': b'channel-movie',
 'data': b'Movie2'}

In [24]:
consumer.get_message()

{'type': 'message',
 'pattern': None,
 'channel': b'channel-movie',
 'data': b'Movie3'}

In [25]:
consumer.get_message()

{'type': 'message',
 'pattern': None,
 'channel': b'channel-cartoon',
 'data': b'Cartoon1'}

In [26]:
consumer.get_message()

{'type': 'message',
 'pattern': None,
 'channel': b'channel-cartoon',
 'data': b'Cartoon2'}

In [27]:
consumer.get_message()

{'type': 'message',
 'pattern': None,
 'channel': b'channel-cartoon',
 'data': b'Cartoon3'}

In [36]:
consumer.get_message()

We must call get_message to know if there is new data in this subscription. But there are better ways

For example using treads, and calling the get_message method in regular intervals

In [29]:
import threading
import time

def reading_by_time(consumer):
    while True:
        message = consumer.get_message()
        if message:
            print(message)
        time.sleep(5)  # be nice to the system :)
        
reader = threading.Thread(target=reading_by_time, args=(consumer,))
reader.start()

Now publish some data and the thread should capture that message and print it after some seconds

In [30]:
redis.publish('channel-cartoon', 'Cartoon3')

1

{'type': 'message', 'pattern': None, 'channel': b'channel-cartoon', 'data': b'Cartoon3'}


In [31]:
redis.publish('channel-movie', 'Movie4')

1

{'type': 'message', 'pattern': None, 'channel': b'channel-movie', 'data': b'Movie4'}


Another way is using the listen method that will halt the execution until a message is received

In [32]:
import threading
import time

def reading_by_listen(consumer):
    for message in consumer.listen():
        print(message)
        
reader = threading.Thread(target=reading_by_listen, args=(consumer,))
reader.start()

In [33]:
redis.publish('channel-cartoon', 'Cartoon3')

1

{'type': 'message', 'pattern': None, 'channel': b'channel-cartoon', 'data': b'Cartoon3'}


In [34]:
redis.publish('channel-movie', 'Movie4')

{'type': 'message', 'pattern': None, 'channel': b'channel-movie', 'data': b'Movie4'}

1

When finished with the consumer, do not forget closing it

In [37]:
consumer.close()

To get the subscribed channels

In [38]:
redis.pubsub_channels()

[]

To get the number of consumers subscribed at every channel. The number will be 0 if the consumer was closed

In [39]:
redis.pubsub_numsub('channel-movie', 'channel-cartoon')
# Return a list of (channel, number of subscribers)

[(b'channel-movie', 0), (b'channel-cartoon', 0)]